# Lab 3: Modelos de clasificación en presencia de clases desbalanceadas


## ¡Bienvenido/a!

Te invitamos a realizar el primer trabajo.
- Objetivo: Para un caso práctico desarrollado en Python, aplicar diferentes técnicas de clasificación a un conjunto de datos con clases desbalanceadas, donde también se aplicarán diferentes alternativas para balancear las clases.
- Tipo de actividad: Individual
- Tipo de evaluación: Sumativa 
- Ponderación: 12%
- Puntaje: 100 puntos
- Calificación: Escala de 1 a 7, con una exigencia de 50%. La nota mínima para aprobar es 4.0.


## Introducción:

El objetivo de este laboratorio es explorar el uso de modelos de clasificación en presencia de clases desbalanceadas.
Para esto, vamos a utilizar la base de datos `post_pabellon.csv`, la cual contiene datos de pacientes que fueron
sometidos a una biopsia (extracción de tejido). La variable de interés es `HOSPITALIZACION`, la cual indica si el
paciente fue hospitalizado luego de la biopsia. 

La siguiente tabla contiene la descripción de las variables (buscar información adicional en caso de considerarlo necesario) de la base de datos:

|Variable|Descripción|Tipo|
|--------|-----------|--|
|EDAD|Edad del paciente| Numérica|
|DIABETES|Indica si el paciente tiene o no diabetes| Binaria 1/0|
|HOSPITALIZACIÓN ULTIMO MES|Indica si el paciente fue hospitalizado el mes previo al procedimiento| Binaria 1/0|
|CUP|Uso de cateter urinario al momento de la biopsia| Binaria 1/0|
|ENF. CRONICA PULMONAR OBSTRUCTIVA|Indica si el paciente tiene una efermedad crónica polmunar obstructuva| Binaria 1/0|
|VOLUMEN PROSTATICO|Incica si el volumen prostatico es mayor a 40 $cm^3$| Binaria 1/0|
|PSA| Concentración del antígeno prostático específico en la sangre| Numérica|
|BIOPSIAS PREVIAS| Indica si el paciente ha tenido biopsias previas| Binaria 1/0|
|ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS| Indica el tipo de antibiótico utilizado en la profilaxis| Categórica nominal |
|NUMERO DE MUESTRAS TOMADAS| Número de muestras tomadas en la biopsia| Numérica|
|BIOPSIA| Resultado de la biopsia| Binaria 1/0 (1: maligno, 0: benigno)|
|HOSPITALIZACION| Indica si el paciente fue hospitalizado luego de la biopsia| Binaria 1/0|


Los modelos que vamos a utilizar son los siguientes:
- Support Vector Classifier (SVC)
- Random Forest Classifier (RFC)
- Naive Bayes Classifier (NBC)

Además, vamos a utilizar las siguientes métricas para evaluar el ajuste y también medir el desempeño de los modelos (capacidad predictiva):
- `Accuracy`: Porcentaje de casos correctamente clasificados sobre el total de casos.
- `Precision`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos clasificados como positivos.
- `Recall`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos positivos.
- `F1-Score`: Media armónica entre Precision y Recall.
- `AUC`: Área bajo la curva ROC.

Por último, para equilibrar las clases vamos a utilizar las siguientes técnicas:
- `RandomOverSampler`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria.
- `RandomUnderSampler`: Técnica de submuestreo que elimina muestras de la clase mayoritaria.
- `NearMiss`: Técnica de submuestreo que elimina muestras de la clase mayoritaria basándonos en la distancia a las muestras de la clase minoritaria.
- `ADASYN`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria basándonos en la densidad de las muestras de la clase minoritaria.
- Combinación de `RandomOverSampler` y `RandomUnderSampler`.

In [1]:
# Módulos básicos para análisis y manipulación de datos
import numpy as np
import pandas as pd

# Modelos de clasificación
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB

# Módulos para evaluación de modelos
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# Módulos para el balanceo de datos
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, ADASYN

1. Cargar la base de datos guardada en el archivo `post_pabellon.xlsx`, y guardar la data en el objeto `datos`. Luego, separar las variables de entrada en `X` y la variable respuesta en `y`. En el caso de las variables de entrada crear variables dummies para las variables categoricas nominales de más de dos niveles, para ello utilice la función `get_dummies` de la librería `pandas` con la opción `drop_first=True`.

In [2]:
datos = None      # Variable que debe modificar
y = None          # Variable que debe modificar
X = None          # Variable que debe modificar

# your code here
datos = pd.read_excel('post_pabellon.xlsx')
X = datos[[col for col in datos.columns if col!='HOSPITALIZACION']].copy()
X = pd.get_dummies(X, columns=["ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS"], drop_first=True)

y = datos['HOSPITALIZACION']

In [3]:
# Test 1 - P1: Prueba de la carga de la base de datos y de la creación de las variables de entrada y respuesta tal como se especifica.

2. Realice una descomposición aleatoria estratificada según la variable respuesta con una proporción de 70% para entrenamiento y 30% para validación. Utilice una semilla de 123 para replicar los resultados.

Debe entregar los siguientes objetos:
- `X_train`: Matriz de predictores para entrenamiento.
- `X_test`: Matriz de predictores para validación.
- `y_train`: Vector de respuesta para entrenamiento.
- `y_test`: Vector de respuesta para validación.

**OBS**: Utilice la función `train_test_split` de la librería `sklearn`.

In [4]:
X_train = None      # Variable que debe modificar
y_train = None      # Variable que debe modificar

X_test = None       # Variable que debe modificar
y_test = None       # Variable que debe modificar

# your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

In [5]:
# Test 1 - P2: Prueba de dimensiones de los datos y aleatoriedad de la separación

3. A continuación, debe realizar una **validación cruzada estratificada** con 5 folds sobre la partición de entrenamiento para comparar diferentes modelos que 
se especificarán más adelante. 

Intrucciones:
- Utilice la función `StratifiedKFold` del módulo `model_selection` de la librería `sklearn`. Este objeto debe ser entregado como argumento a la función `cross_val_score`.
- Se debe utilizar el número 123 como semilla de aleatorización de la función `StratifiedKFold`. Además utilice el argumento `shuffle=True`.
- Se deben evaluar las siguientes métricas de ajuste en cada fold: `accuracy`, `precision`, `recall`, `f1` y `roc_auc`.
- El objetivo es comparar el desempeño de los modelos en cada fold, para esto, se debe calcular la **media** de cada métrica de ajuste por modelo.
- Debe entregar un dataframe de nombre `cv_results` con la siguiente estructura:

    | |Accuracy|Precision|Recall|F1|AUC|
    |-|--------|---------|------|--|-------|
    |SV|-|-|-|-|-|
    |RF|-|-|-|-|-|
    |NB|-|-|-|-|-|

    En python:
        
    ```python
    # Dataframe vacío
    cv_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
    ```
- Los modelos deben considerar todas las variables de la base de datos y utilizar los siguientes hiperparámetros:
    - SVC: `C=15`, `kernel='poly'`, `degree=2`
    - RFC: `n_estimators=50`, `max_depth=10`, `random_state=123`, `max_samples=0.8`, `max_features='log2'`
    - NBC: `class_prior=[0.5, 0.5]`

In [6]:
## Modelos que debe ajustar

SVC_model = SVC(C=15, kernel='poly', degree=2)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

# your code here

cv_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        svc_cv_accuracy = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_train,
            y=y_train,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results.loc[model, metric] = svc_cv_accuracy

cv_results

,Accuracy,Precision,Recall,F1,AUC
SV,0.957215,0.0,0.0,0.0,0.735307
RF,0.959747,0.3,0.133333,0.18,0.855702
NB,0.838829,0.051496,0.183333,0.078941,0.619408


In [7]:
# Test 1 - P3: Se comparan los valores de la matriz cv_results completa.

4. De acuerdo con los resultados anteriores, ¿cuál modelo recomedaría?. 

Para ello primero debe indicar la medida de desempeño en `medida`, donde debe escoger entre 'Accuracy', 'Precision', 'F1', 'Recall' y 'AUC'. (Por ejemplo, `medida='Accuracy'`). Una vez seleccionada la medida debe indicar el modelo de mejor desempeño, de acuerdo con la validación cruzada, en el objeto `model_select`, para indicar el mejor modelo utilizar `SV`, `RF` o `NB` (Por ejemplo, `model_select='SV'`).

In [8]:
medida = None            # Variable que debe modificar
model_select = None      # Variable que debe modificar

# your code here
medida='Recall'
model_select='NB'

In [9]:
# Test 1 - P4: Elección adecuada del modelo

5. También se debe evaluar el desempeño de los modelos, es decir, su capacidad predictiva o de generalización. Tener en cuenta, que por lo general, se debe evaluar solo la generalización del modelo escogido con la muestra de entrenamiento, cuando este último se considera aceptable. Solo con fines comparativos se evaluará la generalización de los 3 modelos propuestos.


Para esto, ajuste los modelos del punto anterior con los datos de entrenamiento y evalúelos con los datos de validación. Debe entregar un dataframe de nombre `test_results` con la siguiente estructura:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

En python:
    
```python
# Dataframe vacío
test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
```

Considere los mismos hiperparámetros del punto anterior. En caso de división por cero se debe deja el indicar (o métrica) en 0, para ello utilice el argumente `zero_division=0`.

In [20]:
# your code here
test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_train, y_train)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_train, y_train)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_train, y_train)


diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            
test_results

,Accuracy,Precision,Recall,F1,AUC
SV,0.959064,0.0,0.0,0.0,0.719512
RF,0.959064,0.0,0.0,0.0,0.891115
NB,0.853801,0.090909,0.285714,0.137931,0.729094


In [11]:
# Test 1 - P5: Se comparan los valores de la matriz test_results completa.

Hasta ahora, hemos trabajado con los datos "originales", es decir, sin balancear las clases. Lo que produce este problema es la inclinación de los modelos a prever la clase mayoritaria, resultando en un rendimiento deficiente. Para solucionar esto, vamos a utilizar las técnicas de remuestreo mencionadas en la introducción del laboratorio.

Además, habrá notado que la métricas: `precision` y `recall` no han tenido un buen desempeño en los modelos anteriores. Esto se debe a que estas métricas están relacionadas con la clase minoritaria. Luego, para mejorar el desempeño de estas métricas, debemos balancear las clases.

Las siguientes preguntas tienen como propósito evaluar el rendimiento de los modelos previos utilizando datos balanceados. **Aunque es posible que cada modelo mejore con ajustes más precisos, el enfoque de este laboratorio es analizar el comportamiento de los modelos al equilibrar las clases con las diferentes técnicas disponibles**.

**MUY IMPORTANTE**: para las preguntas 6, 7 y 8 debe entregar los dataframes con el mismo formato anterior, es decir:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

6 - Realice un sobremuestreo de la clase minoritaria utilizando la técnica `RandomOverSampler` y evalúe el desempeño de los modelos con los datos balanceados. Debe replicar lo solicitado en el punto 3 y 5, pero ahora con los datos balanceados. Debe entregar los siguientes objetos:

- `cv_results_balanced`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='minority'` en `RandomOverSampler`.

In [42]:
# your code here
ros = RandomOverSampler(random_state=123, sampling_strategy='minority')
X_res, y_res = ros.fit_resample(X_train, y_train)

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True, n_splits=5)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        cv_metric = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_res,
            y=y_res,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results_balanced.loc[model, metric] = cv_metric

        
        
SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_res,y_res)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_res,y_res)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_res,y_res)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

test_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results_balanced.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results_balanced.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results_balanced.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            

cv_results_balanced
test_results_balanced

,Accuracy,Precision,Recall,F1,AUC
SV,0.345029,0.058824,1.0,0.111111,0.848432
RF,0.953216,0.428571,0.428571,0.428571,0.857143
NB,0.555556,0.04,0.428571,0.073171,0.642857


In [43]:
# Test 1 - P6: Se comparan los valores de la matriz cv_results_balanced completa.

In [44]:
# Test 2 - P6: Se comparan los valores de la matriz test_results_balanced completa.

7 - Nuevamente realice la validación cruzada y la evaluación del conjunto de prueba para los modelos de clasificación, pero esta vez utilizando la técnica `RandomUnderSampler` para balancear las clases. Debe entregar los siguientes objetos:

- `cv_results_balanced2`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced2`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='majority'` en `RandomUnderSampler`.

In [45]:
# your code here
ros = RandomUnderSampler(random_state=123, sampling_strategy='majority')
X_res, y_res = ros.fit_resample(X_train, y_train)

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True, n_splits=5)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        cv_metric = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_res,
            y=y_res,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results_balanced2.loc[model, metric] = cv_metric

        
        
SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_res,y_res)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_res,y_res)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_res,y_res)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

test_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results_balanced2.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results_balanced2.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results_balanced2.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            

cv_results_balanced2

test_results_balanced2

,Accuracy,Precision,Recall,F1,AUC
SV,0.345029,0.051282,0.857143,0.096774,0.451655
RF,0.777778,0.121951,0.714286,0.208333,0.791812
NB,0.526316,0.059524,0.714286,0.10989,0.748258


In [46]:
# Test 1 - P7: Se comparan los valores de la matriz cv_results_balanced2 completa.

In [47]:
# Test 2 - P7: Se comparan los valores de la matriz test_results_balanced2 completa.

8 - Por último, nuevamente balancear las clases, pero esta vez utilizando las siguientes técnicas de balanceo de clases:
- `NearMiss`
- `ADASYN`
- Combinación de `RandomOverSampler` y `RandomUnderSampler` con `sampling_strategy=0.5` en ambos.

Los nombres de los objetos deben ser los siguientes:
- `cv_results_balanced_nm`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `NearMiss`.
- `test_results_balanced_nm`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `NearMiss`.
- `cv_results_balanced_ad`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `ADASYN`.
- `test_results_balanced_ad`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `ADASYN`.
- `cv_results_balanced_comb`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `RandomOverSampler` y `RandomUnderSampler`.
- `test_results_balanced_comb`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `RandomOverSampler` y `RandomUnderSampler`.

Debe utilizar los siguientes hiperparámetros para cada técnica de balanceo:
- `NearMiss`: `sampling_strategy='majority'`
- `ADASYN`: `random_state=123`, `sampling_strategy='minority'`
- `RandomOverSampler` y `RandomUnderSampler`: `random_state=123`, `sampling_strategy=0.5`

In [48]:
# your code here
ros = NearMiss(sampling_strategy='majority')
X_res, y_res = ros.fit_resample(X_train, y_train)

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True, n_splits=5)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        cv_metric = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_res,
            y=y_res,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results_balanced_nm.loc[model, metric] = cv_metric

        
# Resultados en test
        
        
SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_res,y_res)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_res,y_res)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_res,y_res)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

test_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results_balanced_nm.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results_balanced_nm.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results_balanced_nm.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            

cv_results_balanced_nm
test_results_balanced_nm

,Accuracy,Precision,Recall,F1,AUC
SV,0.362573,0.044643,0.714286,0.084034,0.50784
RF,0.467836,0.053191,0.714286,0.09901,0.665505
NB,0.748538,0.071429,0.428571,0.122449,0.658101


In [49]:
# your code here
ros = ADASYN(random_state=123,sampling_strategy='minority')
X_res, y_res = ros.fit_resample(X_train, y_train)

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True, n_splits=5)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        cv_metric = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_res,
            y=y_res,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results_balanced_ad.loc[model, metric] = cv_metric

        
# Resultados en test
        
        
SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_res,y_res)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_res,y_res)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_res,y_res)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

test_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results_balanced_ad.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results_balanced_ad.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results_balanced_ad.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            

cv_results_balanced_ad
test_results_balanced_ad

,Accuracy,Precision,Recall,F1,AUC
SV,0.438596,0.067961,1.0,0.127273,0.841463
RF,0.865497,0.192308,0.714286,0.30303,0.849739
NB,0.54386,0.050633,0.571429,0.093023,0.635889


In [61]:
# your code here
ros = RandomOverSampler(random_state=123, sampling_strategy=0.5)
rus = RandomUnderSampler(random_state=123, sampling_strategy=0.5)

X_res, y_res = ros.fit_resample(X_train, y_train)
X_res, y_res = rus.fit_resample(X_res, y_res)

SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

skfold = StratifiedKFold(random_state=123, shuffle=True, n_splits=5)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':'accuracy',
    'Precision':'precision',
    'Recall':'recall',
    'F1':'f1',
    'AUC':'roc_auc'
}

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        cv_metric = np.mean(cross_val_score(
            estimator=diccionario_modelos[model],
            X=X_res,
            y=y_res,
            cv=skfold,
            scoring=diccionario_metricas[metric]))
        
        cv_results_balanced_comb.loc[model, metric] = cv_metric

        
# Resultados en test
        
        
SVC_model = SVC(C=15, kernel='poly', degree=2, probability=True).fit(X_res,y_res)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2').fit(X_res,y_res)
NB_model = BernoulliNB(class_prior=[0.5, 0.5]).fit(X_res,y_res)

diccionario_modelos = {
    'SV':SVC_model,
    'RF':RF_model,
    'NB':NB_model
}


diccionario_metricas = {
    'Accuracy':metrics.accuracy_score,
    'Precision':metrics.precision_score,
    'Recall':metrics.recall_score,
    'F1':metrics.f1_score,
    'AUC':metrics.roc_auc_score
}

test_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for model in ['SV', 'RF', 'NB']:
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
        
        if model=='SV':
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
            
        else:        
            y_pred_test_prob = diccionario_modelos[model].predict_proba(X_test)[:, 1]
            y_pred_test = diccionario_modelos[model].predict(X_test)
        
        if metric in ['Precision', 'Recall', 'F1']:
            test_results_balanced_comb.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test, zero_division=0)
        elif metric=='AUC':
            test_results_balanced_comb.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test_prob)
        else:
            test_results_balanced_comb.loc[model, metric] = diccionario_metricas[metric](y_test,y_pred_test)
            

cv_results_balanced_comb
test_results_balanced_comb

,Accuracy,Precision,Recall,F1,AUC
SV,0.923977,0.125,0.142857,0.133333,0.662892
RF,0.947368,0.416667,0.714286,0.526316,0.902439
NB,0.561404,0.052632,0.571429,0.096386,0.691638


In [51]:
# Test 1 - P8: NearMiss (se comparan los valores de las matrices cv_results_balanced_nm y test_results_balanced_nm)

In [52]:
# Test 2 - P8: ADASYN (se comparan los valores de las matrices cv_results_balanced_ad y test_results_balanced_ad)

In [53]:
# Test 3 - P8: Combinación de técnicas oversampling y undersampling (se comparan los valores de las matrices cv_results_balanced_comb y test_results_balanced_comb)

9. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de entrenamiento. 

El nombre del modelo dede ser `best_model_balancing` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing=1`

In [54]:
best_model_balancing = None    # Objeto que debe modificar
best_balancing = None          # Objeto que debe modificar

# your code here
#print(cv_results_balanced)
#print(cv_results_balanced2)
#print(cv_results_balanced_nm)
#print(cv_results_balanced_ad)
#print(cv_results_balanced_comb)
best_model_balancing='RF'
best_balancing=1

In [55]:
# Test 1 - P9

10. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de test. (Esta comparación es solo confines de comprender la diferencia entre los errores cometidos en la muestra train y test, no se debe seleccionar el modelo desde la muestra test) 

El nombre del modelo dede ser `best_model_balancing_tets` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing_test='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing_test` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing_test=1`

In [56]:
best_model_balancing_test = None    # Objeto que debe modificar
best_balancing_test = None          # Objeto que debe modificar

# your code here
best_model_balancing_test = 'RF'    # Objeto que debe modificar
best_balancing_test = 5          # Objeto que debe modificar

In [57]:
# Test 1 - P10

11. Por último, responda las siguientes preguntas, en ambas considere como métrica el `F1-Score`:

a. El mejor método (modelo de clasificación y técnica de balanceo) con la muestra de entrenamiento coincide con el mejor de la muestra test. Para esto, el objeto `resp_a` debe contener algunas de las siguientes:
- 'Los métodos coinciden'
- 'Los métodos coinciden solo en el modelo pero no en la técnica de balanceo'
- 'Los métodos coinciden solo en la técnica de balanceo pero no en el modelo'
- 'Los métodos no coinciden'

Por ejemplo: `resp_a='Los métodos coinciden'`

b. ¿Cuál de las dos métodologias, utilizando los datos originales o utilizando los datos balanceados, tiene mejor generalización en la muestra test. Para esto, el objeto `resp_b` debe contener algunas de las siguientes:
- 'Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'
- 'Utilizando los datos originales tiene mejores resultados'
- 'Utilizando los datos balanceados tiene mejores resultados'

Por ejemplo: `resp_b='Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'`

In [58]:
print("Con datos de entrenamiento...")
print(cv_results_balanced)
print(cv_results_balanced2)
print(cv_results_balanced_nm)
print(cv_results_balanced_ad)
print(cv_results_balanced_comb)


print("\n\n\nCon datos de testeo...")


print(test_results_balanced)
print(test_results_balanced2)
print(test_results_balanced_nm)
print(test_results_balanced_ad)
print(test_results_balanced_comb)



print("\n\n\nDatos de test originales")

print(test_results)

Con datos de entrenamiento...
    Accuracy Precision    Recall        F1       AUC
SV  0.651316  0.589468       1.0  0.741624  0.691118
RF  0.988158  0.976954       1.0   0.98832  0.998979
NB  0.664474  0.650265  0.718421  0.682299  0.757185
    Accuracy Precision    Recall        F1       AUC
SV  0.466667  0.419048  0.483333  0.415238       0.4
RF  0.609524  0.497619  0.733333  0.591169  0.616667
NB   0.37619      0.42  0.516667  0.450794  0.219444
    Accuracy Precision    Recall        F1       AUC
SV  0.609524  0.616667  0.466667  0.496667  0.694444
RF  0.638095  0.733333       0.4  0.484762  0.752778
NB  0.466667       0.3       0.1  0.146667  0.469444
    Accuracy Precision    Recall        F1       AUC
SV  0.655779   0.59895   0.95581  0.736253  0.767526
RF  0.948916  0.919084  0.987013  0.951299  0.995289
NB  0.761748  0.704358  0.908852  0.792983  0.824141
    Accuracy Precision    Recall        F1       AUC
SV  0.677193  0.687013  0.126316  0.203222  0.745118
RF  0.989474  0.

In [59]:
resp_a = None                         # Objeto que debe modificar
resp_b = None                         # Objeto que debe modificar

# your code here
resp_a = 'Los métodos coinciden solo en el modelo pero no en la técnica de balanceo'
resp_b = 'Utilizando los datos balanceados tiene mejores resultados'

In [ ]:
# Test 1 - P11